In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("domainadaptationexperiments.csv").dropna()
df = pd.concat([df[df['training_method'] == 'FedSCDepth(Retrain)'], df[df['training_method'] == 'FedSCDepth']])

In [2]:
df_full = df[df['test_region_type'] == 'full'].drop(columns=['test_region_type'])

In [3]:
df_after = df_full[df_full['adaptation_state'] == 'after_adaptation'].drop(columns=['adaptation_state'])

In [18]:
group_columns = ['source_train_dataset', 'target_train_dataset', 'training_method']
avg_columns = ['test_abs_rel', 'test_sq_rel', 'test_log10', 'test_rmse', 'test_rmse_log', 'test_a1', 'test_a2', 'test_a3']
df_stability = df_after.groupby(group_columns, as_index=False)[avg_columns].mean()
df_stability

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.1665,2.8950,0.079,13.0960,0.2765,0.7555,0.9025,0.949
1,DDAD,KITTI,FedSCDepth(Retrain),0.1675,2.8315,0.080,12.9700,0.2765,0.7495,0.9040,0.950
2,KITTI,DDAD,FedSCDepth,0.1875,2.8360,0.079,12.3210,0.2700,0.7445,0.9050,0.959
3,KITTI,DDAD,FedSCDepth(Retrain),0.1725,2.5870,0.077,12.3315,0.2645,0.7540,0.9120,0.961


In [5]:
df_before = df_full[df_full['adaptation_state'] == 'before_adaptation'].drop(columns=['adaptation_state'])
#df_before

In [6]:
df_before_plasticity = df_before[df_before['test_dataset'] == df_before['source_train_dataset']]
#df_before_plasticity

In [7]:
df_after_plasticity = df_after[df_after['test_dataset'] == df_after['target_train_dataset']]
#df_after_plasticity

In [17]:
df_plasticity = pd.concat([df_before_plasticity, df_after_plasticity])
group_columns = ['source_train_dataset', 'target_train_dataset', 'training_method']
avg_columns = ['test_abs_rel', 'test_sq_rel', 'test_log10', 'test_rmse', 'test_rmse_log', 'test_a1', 'test_a2', 'test_a3']
df_plasticity = df_plasticity.groupby(group_columns, as_index=False)[avg_columns].mean()
df_plasticity

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.2415,4.7435,0.1400,15.3635,0.4640,0.6505,0.7900,0.8420
1,DDAD,KITTI,FedSCDepth(Retrain),0.2415,4.7300,0.1405,15.3760,0.4645,0.6495,0.7895,0.8420
2,KITTI,DDAD,FedSCDepth,0.1380,2.0975,0.0635,11.1050,0.2290,0.8210,0.9340,0.9685
3,KITTI,DDAD,FedSCDepth(Retrain),0.1475,2.3660,0.0685,12.0320,0.2445,0.7975,0.9240,0.9630


In [9]:
def compute_stability_plasticity_tradeoff(group):
    product_series = {col: np.prod(group[col]) for col in avg_columns}
    product_series_multiplied_by_two = {col: val * 2 for col, val in product_series.items()}
    sum_series = {col: np.sum(group[col]) for col in avg_columns}
    product_series_multiplied_by_two = pd.Series(product_series_multiplied_by_two)
    sum_series = pd.Series(sum_series)
    return product_series_multiplied_by_two / sum_series

In [16]:
df_stability_plasticity = pd.concat([df_stability, df_plasticity])
df_stability_plasticity

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.1665,2.8950,0.0790,13.0960,0.2765,0.7555,0.9025,0.9490
1,DDAD,KITTI,FedSCDepth(Retrain),0.1675,2.8315,0.0800,12.9700,0.2765,0.7495,0.9040,0.9500
2,KITTI,DDAD,FedSCDepth,0.1875,2.8360,0.0790,12.3210,0.2700,0.7445,0.9050,0.9590
3,KITTI,DDAD,FedSCDepth(Retrain),0.1725,2.5870,0.0770,12.3315,0.2645,0.7540,0.9120,0.9610
0,DDAD,KITTI,FedSCDepth,0.2415,4.7435,0.1400,15.3635,0.4640,0.6505,0.7900,0.8420
1,DDAD,KITTI,FedSCDepth(Retrain),0.2415,4.7300,0.1405,15.3760,0.4645,0.6495,0.7895,0.8420
2,KITTI,DDAD,FedSCDepth,0.1380,2.0975,0.0635,11.1050,0.2290,0.8210,0.9340,0.9685
3,KITTI,DDAD,FedSCDepth(Retrain),0.1475,2.3660,0.0685,12.0320,0.2445,0.7975,0.9240,0.9630


In [11]:
df_stability_plasticity_tradeoff = df_stability_plasticity.groupby(['source_train_dataset','target_train_dataset','training_method'], as_index=False).apply(compute_stability_plasticity_tradeoff)
df_stability_plasticity_tradeoff

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.197107,3.595584,0.101005,14.139419,0.346512,0.699079,0.842511,0.892304
1,DDAD,KITTI,FedSCDepth(Retrain),0.197806,3.542418,0.101950,14.070890,0.346651,0.695926,0.842879,0.892746
2,KITTI,DDAD,FedSCDepth,0.158986,2.411477,0.070407,11.681440,0.247816,0.780881,0.919271,0.963727
3,KITTI,DDAD,FedSCDepth(Retrain),0.159023,2.471570,0.072502,12.179909,0.254107,0.775140,0.917961,0.961999


In [12]:
df_stability_ciclic = df_stability.groupby(['training_method'], as_index=False)[avg_columns].mean()
#df_stability_ciclic

In [13]:
df_plasticity_ciclic = df_plasticity.groupby(['training_method'], as_index=False)[avg_columns].mean()
#df_plasticity_ciclic

In [20]:
df_stability_plasticity_ciclic = pd.concat([df_stability_ciclic, df_plasticity_ciclic])
#df_stability_plasticity_ciclic

In [15]:
df_stability_plasticity_ciclic_tradeoff = df_stability_plasticity_ciclic.groupby(['training_method'], as_index=False).apply(compute_stability_plasticity_tradeoff)
df_stability_plasticity_ciclic_tradeoff.sort_values(by=['test_abs_rel'])

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.181427,3.072410,0.089653,13.156329,0.306855,0.737355,0.881631,0.928244
0,FedSCDepth,0.183153,3.118499,0.088943,12.966048,0.305546,0.742807,0.882381,0.928986
